# 3. RAG 구축하기
## 순서
1. 데이터 읽기(`document_loaders`)
2. 문장 분할(`Splitter`)
3. 임베딩(`Embeddings`)
4. 저장(`VectorStore`)
---
5. 검색기(`Retriever`)
6. 검색 성능 개선(`Ensemble Retriever`)
   

## 3.1. 데이터 읽기 - TextLoader사용하기

### 3.1 TextLoader 사용

In [3]:
from langchain_community.document_loaders import TextLoader
import glob

directory_path = "./dataset/preprocessed_text/*.txt" 

# 모든 텍스트 파일 경로 가져오기
file_paths = glob.glob(directory_path)
all_data = []

for file_path in file_paths:
    loader = TextLoader(file_path, encoding='utf-8')
    data = loader.load()
    all_data.extend(data)

In [4]:
all_data

[Document(page_content='직장 내 장애인 인식개선 교육 가이드라인 4: 무료교육 지원\n\n강사지원 및 이러닝 교육\n\n한국장애인고용공단은 교육 실시로 인한 사업주 및 근로자의 부담을 경감하기 위하여 무료로 강사지원 및 이러닝 교육을 실시하고 있습니다.\n\n직장 내 장애인 인식개선 교육 포털 (edu.kead.or.kr) > 무료교육지원 > 수행기관 찾기\n\n신청방법: 상시근로자 5인 이상 300인 미만을 고용하고 있는 사업장에는 무료 강사를 지원해드리고 있습니다.\n\n강사지원: 직장 내 장애인 인식개선 교육 포털(edu.kead.or.kr) > 이러닝 교육 > 교육신청\n\n신청방법: ※ 교육 대상자의 개별 회원가입이 필요하며, 로그인 후 교육 수강이 가능합니다. 다만, 매월 학습 가능 인원이 정해져 있습니다.\n\n복직자 및 신규 입사자 등 회사에서 교육을 실시 하였지만 추가 교육이 필요한 경우, 파견근로자 및 현장근로자 등 사업장 내에서 교육 수강이 어려운 경우에 교육 신청을 요청드립니다.\n\n교육 실시와 관련하여 문의사항은 한국장애인고용공단 대표번호 1588-1519로 문의하시면 친절하게 안내드리겠습니다.\n\n사업주와 근로자는 직장 내 장애인 인식개선을 위한 교육을 로그인하여 수강할 수 있습니다.\n\n이러닝 교육이 제공됩니다.\n\n모든 사업주와 근로자는 직장 내 장애인 인식개선 교육을 연 1회, 1시간 이상 받아야 합니다.\n\n관련 법령: 「장애인고용촉진 및 직업재활법」 제5조의2입니다.\n\n동법 제86조(과태료)\n\n전문강사 교육(집합교육)\n\n교육기관을 통한 교육(집합 및 온라인 교육 등)\n\n사업장 자체적으로 실시하는 교육\n\n무료 교육 지원을 통한 교육\n\n교육 실시 방법은 4가지로 구분할 수 있습니다.\n\n사업체 및 근무 특성에 따라 교육 실시 방법을 선택하시면 됩니다.\n\n전문강사를 초빙하세요: 한국장애인고용공단의 직장 내 장애인 인식개선 교육 강사양성\n\n과정 자격증을 취득한 전문 강사를 통

## 1.2 모델 불러오기

In [5]:
from langchain_community.chat_models import ChatOllama   
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager
from langchain import LLMChain

"""
<hyper parameter explanation>
- temperature : control of randomness(probability)
- top_p : cdf(누적확률분포) 컷팅 비율 (상위 몇프로 사용할 지)
- top_k : 답변으로 뽑아낼 샘플 개수
"""

llm = ChatOllama(
    model="EEVE-Korean-10.8B:latest",
    temperature=0.5,
    # top_p=0,
    top_k=1,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

## 2. 분할(Splitter)

`chunk size`와 `overlap`은 문서마다 최적의 값이 문서마다 다를 수 있으니 최적의 값은 테스트 필요  
`RecursiveJsonSplitter`, `CharacterTextSplitter`, `RecursiveCharacterTextSplitter` 등이 있음  

### RecursiveCharacterTextSplitter
- "\n\n", "\n", " ", "" 재귀적으로 청킹

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pprint import pprint as pp

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    )

splits = text_splitter.split_documents(all_data)
print(f'Splited document count: {len(splits)}')
print(f'len(splits[0].page_content) : {len(splits[0].page_content)}')
pp(splits[0].page_content)

Splited document count: 1001
len(splits[0].page_content) : 243
('직장 내 장애인 인식개선 교육 가이드라인 4: 무료교육 지원\n'
 '\n'
 '강사지원 및 이러닝 교육\n'
 '\n'
 '한국장애인고용공단은 교육 실시로 인한 사업주 및 근로자의 부담을 경감하기 위하여 무료로 강사지원 및 이러닝 교육을 실시하고 있습니다.\n'
 '\n'
 '직장 내 장애인 인식개선 교육 포털 (edu.kead.or.kr) > 무료교육지원 > 수행기관 찾기\n'
 '\n'
 '신청방법: 상시근로자 5인 이상 300인 미만을 고용하고 있는 사업장에는 무료 강사를 지원해드리고 있습니다.')


## 3. 임베딩(Embedding)
텍스트를 벡터화 시켜 의미별 유사도 계산을 할 수 있게 해줌

In [7]:
"""
pip install -U langchain-huggingface
"""
from langchain_huggingface import HuggingFaceEmbeddings
import torch

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

# 임베딩 모델 설정
embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': device}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

c:\Users\dm705\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\dm705\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
# Document.page_content 내용만 담은 리스트 생성
splits_list = list(map(lambda x:x.page_content, splits))
splits_list[:5]

['직장 내 장애인 인식개선 교육 가이드라인 4: 무료교육 지원\n\n강사지원 및 이러닝 교육\n\n한국장애인고용공단은 교육 실시로 인한 사업주 및 근로자의 부담을 경감하기 위하여 무료로 강사지원 및 이러닝 교육을 실시하고 있습니다.\n\n직장 내 장애인 인식개선 교육 포털 (edu.kead.or.kr) > 무료교육지원 > 수행기관 찾기\n\n신청방법: 상시근로자 5인 이상 300인 미만을 고용하고 있는 사업장에는 무료 강사를 지원해드리고 있습니다.',
 '강사지원: 직장 내 장애인 인식개선 교육 포털(edu.kead.or.kr) > 이러닝 교육 > 교육신청\n\n신청방법: ※ 교육 대상자의 개별 회원가입이 필요하며, 로그인 후 교육 수강이 가능합니다. 다만, 매월 학습 가능 인원이 정해져 있습니다.\n\n복직자 및 신규 입사자 등 회사에서 교육을 실시 하였지만 추가 교육이 필요한 경우, 파견근로자 및 현장근로자 등 사업장 내에서 교육 수강이 어려운 경우에 교육 신청을 요청드립니다.',
 '교육 실시와 관련하여 문의사항은 한국장애인고용공단 대표번호 1588-1519로 문의하시면 친절하게 안내드리겠습니다.\n\n사업주와 근로자는 직장 내 장애인 인식개선을 위한 교육을 로그인하여 수강할 수 있습니다.\n\n이러닝 교육이 제공됩니다.\n\n모든 사업주와 근로자는 직장 내 장애인 인식개선 교육을 연 1회, 1시간 이상 받아야 합니다.\n\n관련 법령: 「장애인고용촉진 및 직업재활법」 제5조의2입니다.\n\n동법 제86조(과태료)\n\n전문강사 교육(집합교육)\n\n교육기관을 통한 교육(집합 및 온라인 교육 등)\n\n사업장 자체적으로 실시하는 교육',
 '교육기관을 통한 교육(집합 및 온라인 교육 등)\n\n사업장 자체적으로 실시하는 교육\n\n무료 교육 지원을 통한 교육\n\n교육 실시 방법은 4가지로 구분할 수 있습니다.\n\n사업체 및 근무 특성에 따라 교육 실시 방법을 선택하시면 됩니다.\n\n전문강사를 초빙하세요: 한국장애인고용공단의 직장 내 장애인 인식개선

In [17]:
# embedding ㅇ
embedding = embedding_model.embed_documents(splits_list)

## 4. 저장(`VectorStore`)

In [30]:
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

vectorstore = FAISS.from_documents(splits,
                                   embedding = embedding_model,)

# 로컬에 DB 저장
MY_FAISS_INDEX = "MY_FAISS_INDEX"
vectorstore.save_local(MY_FAISS_INDEX)

### 로컬 VectorDB 사용 방법

In [105]:
# 로컬 DB 불러오기
# vectorstore = FAISS.load_local(MY_FAISS_INDEX, 
#                                embedding_model, 
#                                allow_dangerous_deserialization=True # 잠재적으로 위험한 데이터 구조나 객체를 포함할 수 있는 인덱스 파일의 로딩을 허용. 주로 자신이 직접 생성하고 저장한 인덱스 파일을 로드할 때 사용
#                                )

# retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # 유사도 높은 5문장 추출

## 5. 검색(`Retriever`)

In [35]:
# 6의 검색성능 개선 부분으로 넘김
pass

## 6. 검색성능 개선(`Ensemble`)
- 형태소 분석기 + BM25, FAISS 리트리버로 한글 검색 성능 개선

In [25]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

In [26]:
temp_doc = splits_list[0]
temp_doc

'직장 내 장애인 인식개선 교육 가이드라인 4: 무료교육 지원\n\n강사지원 및 이러닝 교육\n\n한국장애인고용공단은 교육 실시로 인한 사업주 및 근로자의 부담을 경감하기 위하여 무료로 강사지원 및 이러닝 교육을 실시하고 있습니다.\n\n직장 내 장애인 인식개선 교육 포털 (edu.kead.or.kr) > 무료교육지원 > 수행기관 찾기\n\n신청방법: 상시근로자 5인 이상 300인 미만을 고용하고 있는 사업장에는 무료 강사를 지원해드리고 있습니다.'

In [32]:
print(" ".join(kiwi_tokenize(temp_doc)))
print(kiwi_tokenize(temp_doc))

직장 내 장애인 인식 개선 교육 가이드라인 4 : 무료 교육 지원 강사 지원 및 이 러닝 교육 한국장애인고용공단 은 교육 실시 로 인 하 ᆫ 사업주 및 근로자 의 부담 을 경감 하 기 위하 어 무료 로 강사 지원 및 이 러닝 교육 을 실시 하 고 있 습니다 . 직장 내 장애인 인식 개선 교육 포털 ( edu.kead.or.kr ) > 무료 교육 지원 > 수행 기관 찾 기 신청 방법 : 상시 근로자 5 인 이상 300 이 ᆫ 미만 을 고용 하 고 있 는 사업장 에 는 무료 강사 를 지원 하 어 드리 고 있 습니다 .
['직장', '내', '장애인', '인식', '개선', '교육', '가이드라인', '4', ':', '무료', '교육', '지원', '강사', '지원', '및', '이', '러닝', '교육', '한국장애인고용공단', '은', '교육', '실시', '로', '인', '하', 'ᆫ', '사업주', '및', '근로자', '의', '부담', '을', '경감', '하', '기', '위하', '어', '무료', '로', '강사', '지원', '및', '이', '러닝', '교육', '을', '실시', '하', '고', '있', '습니다', '.', '직장', '내', '장애인', '인식', '개선', '교육', '포털', '(', 'edu.kead.or.kr', ')', '>', '무료', '교육', '지원', '>', '수행', '기관', '찾', '기', '신청', '방법', ':', '상시', '근로자', '5', '인', '이상', '300', '이', 'ᆫ', '미만', '을', '고용', '하', '고', '있', '는', '사업장', '에', '는', '무료', '강사', '를', '지원', '하', '어', '드리', '고', '있', '습니다', '.']


### KR 형태소 분석기(kiwi) + BM25, FAISS 리트리버 앙상블

In [75]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

bm25 = BM25Retriever.from_documents(splits)
kiwi_bm25 = BM25Retriever.from_documents(splits, preprocess_func=kiwi_tokenize)

# search_type = 'similarity'외에 'mmr'로도 비교해야 함.
faiss = FAISS.from_documents(splits, embedding_model).as_retriever(search_type='similarity')
# faiss = vectorstore.as_retriever()

# # bm25 성능이 구려서, faiss만 쓰는것도 생각해봐야 할 듯
# bm25_faiss_73 = EnsembleRetriever(
#     retrievers=[bm25, faiss],
#     weights=[0.7, 0.3],
#     search_type="mmr",
# )

kiwibm25_faiss_37 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],
    weights=[0.3, 0.7],
    search_type="mmr",
)

kiwibm25_faiss_73 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],
    weights=[0.7, 0.3],
    search_type="mmr",
)

In [76]:
# 앙상블 리트리버
retrievers = {
    "kiwibm25_faiss_73": kiwibm25_faiss_73,
    "kiwibm25_faiss_37" : kiwibm25_faiss_37,
}

In [114]:
def print_search_results(retrievers, query):
    global llm

    print(f"Query: {query}")
    print("==="* 20)
    answer_docs = []

    template = """
    당신은 유능하지만 창의성을 좋아하는 사람입니다.
    사용자의 물음에 근거와 함께 답변해주세요.
    질문: {question}
    문서: {context}
    """
    prompt = ChatPromptTemplate.from_template(template)

    # 3. 출력 파서 설정하기
    output_parser = StrOutputParser()
    

    for name, retriever in retrievers.items():
        # 리트리버를 통한 문서 검색
        retrieved_docs = retriever.invoke(query)
        
        # 검색된 문서 내용을 하나로 합침
        context = "\n".join([doc.page_content for doc in retrieved_docs])
        llm_chain = prompt | llm | output_parser

        response = llm_chain.invoke({'question': query, 'context': context})
        print(response)
        print("==="* 20)
        print('근거 자료')
        print(f"{name}:\n\n{retriever.invoke(query)[-1].page_content}")
        print("==="* 20)
        
    return response, answer_docs

In [115]:
answers = print_search_results(retrievers, query='장애인은 선천적으로 타고나는 건가요?')
# faiss retriever 성능이 bm25보다 더 좋다보니 결과가 아래와같이 나오는 것 같다.


Query: 장애인은 선천적으로 타고나는 건가요?
질문: 장애인 중 선천적 장애인이 많습니까, 후천적 장애인이 많습니까?

답변: 일반적으로 장애인의 약 90%가 교통사고나 질병과 같은 환경적인 요인이나 사고로 인해 발생하는 후천적 장애를 가지고 있습니다.

신뢰도: 95%

설명:
장애는 선천적으로 타고나는 경우도 있지만, 대부분의 경우 후천적으로 발생합니다. 세계보건기구(WHO)에 따르면 전 세계적으로 약 10억 명의 사람들이 다양한 형태의 장애를 가지고 있으며, 이 중 약 80-90%가 후천적 장애를 가지고 있습니다.

후천적 장애의 주요 원인으로는 다음과 같은 것들이 있습니다:
* 선천적 장애: 다운증후군이나 척추이분증과 같이 태어날 때부터 발생하는 장애입니다.
* 사고나 부상: 교통사고, 낙상, 화재와 같은 사고가 후천적 장애를 일으킬 수 있습니다.
* 질병: 당뇨병이나 파킨슨병과 같은 만성 질환도 후천적 장애로 이어질 수 있습니다.

장애의 유병률은 국가마다 다르며, 선진국에서는 인구 고령화로 인해 장애율이 더 높습니다. 또한, 개발 도상국에서는 의료 서비스의 부족으로 인해 장애인이 제대로 된 치료를 받지 못하는 경우가 많아 장애율이 더 높을 수 있습니다.

장애는 선천적일 수도 있고 후천적일 수도 있지만, 중요한 것은 모든 사람이 장애를 가지고 있거나 장애가 있는 사람을 돌보는 책임이 있다는 것입니다. 장애인은 사회의 일부이며 존중과 존엄성을 받아야 합니다.

신뢰도: 95%질문: 장애인 중 선천적 장애인이 많습니까, 후천적 장애인이 많습니까?

답변: 일반적으로 장애인의 약 90%가 교통사고나 질병과 같은 환경적인 요인이나 사고로 인해 발생하는 후천적 장애를 가지고 있습니다.

신뢰도: 95%

설명:
장애는 선천적으로 타고나는 경우도 있지만, 대부분의 경우 후천적으로 발생합니다. 세계보건기구(WHO)에 따르면 전 세계적으로 약 10억 명의 사람들이 다양한 형태의 장애를 가지고 있으며, 이 중 약 80-90%가 후천적 장애를 가지고 있습니다.



In [118]:
pp(answers)

[' 장애인 여부를 선천적으로 타고나는지에 대한 질문과 관련하여, 장애는 다양한 원인과 유형이 있으며, 일부 장애는 선천적일 수 있지만 많은 '
 '경우 후천적입니다.\r\n'
 '\r\n'
 '신체적 장애의 경우, 선천적 장애인은 드물고 대부분 후천적인 사고나 질병으로 인해 발생합니다. 예를 들어 교통사고, 산업재해 또는 '
 '뇌졸중과 같은 질환으로 인한 신체적 손상이 있습니다.\r\n'
 '\r\n'
 '정신적 장애는 선천적일 수 있지만, 자폐증이나 다운 증후군과 같이 유전적으로 전달되는 경우도 있고, 조산이나 임신 중 약물 노출로 인한 '
 '후천적인 경우도 있습니다.\r\n'
 '\r\n'
 '장애의 유병률은 국가마다 다르며, 세계보건기구(WHO)에 따르면 전 세계적으로 약 15%의 인구가 장애를 가지고 있으며, 이 중 대부분은 '
 '후천적입니다.\r\n'
 '\r\n'
 '요약하자면, 일부 장애는 선천적일 수 있지만 대부분의 장애는 후천적이며 사고나 질병으로 인해 발생합니다.',
 []]


In [116]:
res = answers[0]
pp(res)

(' 장애인 여부를 선천적으로 타고나는지에 대한 질문과 관련하여, 장애는 다양한 원인과 유형이 있으며, 일부 장애는 선천적일 수 있지만 많은 '
 '경우 후천적입니다.\r\n'
 '\r\n'
 '신체적 장애의 경우, 선천적 장애인은 드물고 대부분 후천적인 사고나 질병으로 인해 발생합니다. 예를 들어 교통사고, 산업재해 또는 '
 '뇌졸중과 같은 질환으로 인한 신체적 손상이 있습니다.\r\n'
 '\r\n'
 '정신적 장애는 선천적일 수 있지만, 자폐증이나 다운 증후군과 같이 유전적으로 전달되는 경우도 있고, 조산이나 임신 중 약물 노출로 인한 '
 '후천적인 경우도 있습니다.\r\n'
 '\r\n'
 '장애의 유병률은 국가마다 다르며, 세계보건기구(WHO)에 따르면 전 세계적으로 약 15%의 인구가 장애를 가지고 있으며, 이 중 대부분은 '
 '후천적입니다.\r\n'
 '\r\n'
 '요약하자면, 일부 장애는 선천적일 수 있지만 대부분의 장애는 후천적이며 사고나 질병으로 인해 발생합니다.')


In [104]:
pp(answers[0])

[Document(page_content='모든 시각장애인이 점자를 읽고 사용할 수 있다는 생각인데요, 이 두 가지 모두 그렇지 않느냐고 물으신다면 대답은...\n\n옛날에는 선천적인 이유가 있는 시각장애인이 많았습니다.\n\nNo!!\n\n현대사회에서는 물론 선천적인 원인도 있지만 환경적인 요인과 사고 등으로 인하여 후천적으로 시각장애를 갖게 되는 경우도 매우 많습니다.', metadata={'source': './dataset/preprocessed_text\\data8_preprocessed_chunk20.txt'}),
 Document(page_content='환자: 28군데 요양병원에서 모두 감염환자 입원을 거부하는 건가요? 우리가 등록장애인이 아니기 때문인가요?\n\n아니요, 하지만 이는 명백한 차별입니다. 우리는 어디로 가야 하나요?\n\n이 환자들은 입원 거부행위는 장애인 차별금지법상 차별행위라고 보고 요양병원과 이들에 대한 책임을 미루는 정부 부처를 국가인권위원회에 진정했습니다.\n\nHIV/AIDS 감염인 역시 장애인복지법상 등록장애인이 아니지만, 감염 후 10년 이상 경과하면 면역력 저하로 여러 가지 증상이 나타날 수 있으며, 이러한 증상들로 인해', metadata={'source': './dataset/preprocessed_text\\data9_preprocessed_chunk10.txt'}),
 Document(page_content='과거 특정 지자체에서 쓰레기봉투를 집으로 배달해주는 사업을 운영한 적이 있습니다.\n\n관련 사업이 없어졌음에도 불구하고 지금까지도 요구가 계속 있는 것으로 보입니다. 공무원은 순환보직으로 근무하는 방식이기 때문에 관련 사실을 알지 못해 난감한 경우가 있었을 텐데요. 이때는 일시적 사업이었습니다.\n\n로 진행되었음을 알려드리고 이해를 구하는 것이 좋을 것으로 생각됩니다.\n\n68\n\n장애인 인식개선 가이드\n\n대체 어디에 문의해야 하는 건가요?\n\n9.\n\n?', metadata={'sour

In [111]:
retrievers

{'kiwibm25_faiss_73': EnsembleRetriever(retrievers=[BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x00000269F891F3D0>, preprocess_func=<function kiwi_tokenize at 0x00000269FA9F6520>), VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000269F851AD50>)], weights=[0.7, 0.3]),
 'kiwibm25_faiss_37': EnsembleRetriever(retrievers=[BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x00000269F891F3D0>, preprocess_func=<function kiwi_tokenize at 0x00000269FA9F6520>), VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000269F851AD50>)], weights=[0.3, 0.7])}

In [51]:
from langchain import LLMChain


template = """
다음 질문을 듣고 물음에 답해주세요.
{text}
"""
prompt = ChatPromptTemplate.from_template(template)

# 3. 출력 파서 설정하기
output_parser = StrOutputParser()

input_text = "장애 유형은 몇 가지가 있나요?"
# 4. LLMChain 만들기
llm_chain = LLMChain(llm=llm, prompt=prompt, output_parser=output_parser)
result = llm_chain.invoke({"text": input_text})


질문에 답변드리자면, 장애는 신체적, 인지적, 정신적, 감각적 등 다양한 형태로 나타날 수 있습니다. 세계보건기구(WHO)에 따르면 전 세계적으로 약 10억 명의 사람들이 장애를 가지고 있으며, 이는 전 세계 인구의 약 15%에 해당합니다.

장애 유형은 다음과 같습니다:

* **신체적 장애:** 신체적 장애는 이동성 제한, 시력 또는 청력 손실, 만성 질환과 같은 신체 기능의 손상으로 인해 발생합니다.
* **인지적 장애:** 인지적 장애는 지적 능력이나 학습 능력의 결핍으로 인해 발생합니다. 예를 들어 자폐증, 다운 증후군, 주의력 결핍 과잉행동장애(ADHD) 등이 있습니다.
* **정신적 장애:** 정신적 장애는 정신 질환 또는 심리적 트라우마로 인해 발생합니다. 예를 들면 우울증, 불안 장애, 외상 후 스트레스 장애(PTSD) 등이 있습니다.
* **감각적 장애:** 감각적 장애는 청력 손실이나 시각 손상과 같은 감각 기능의 손상으로 인해 발생합니다.

장애 유형은 다양하며 각 개인에게 미치는 영향도 다릅니다. 어떤 사람들은 경미한 장애를 가지고 있어 일상 생활에 큰 영향을 받지 않을 수 있지만, 다른 이들은 중대한 장애를 가지고 있어 상당한 지원과 적응이 필요할 수 있습니다.

장애는 사람의 삶에 큰 영향을 미칠 수 있는 복잡한 문제입니다. 장애가 있는 사람들과 그들의 가족은 종종 도전과제에 직면하며, 이러한 도전을 극복하기 위해 자원과 지원이 필요합니다.

장애 유형을 이해하는 것은 장애가 있는 사람들을 이해하고 지원하는 데 있어 중요한 단계입니다. 장애의 다양한 형태를 인식함으로써 우리는 더 포괄적이고 접근 가능한 사회를 만들 수 있습니다.